In [69]:
import pandas as pd
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.model_selection import train_test_split

In [43]:
autumn_data = pd.read_csv('autumn_data.csv')
autumn_data.shape

(2201, 4)

In [45]:
autumn_data.head()

,datetime,active_electricity,outdoor_temperature,indoor_temperature
0,01-10-22 3:00,693.1875,27.769607,21.416667
1,01-10-22 4:00,460.0375,27.924395,21.083333
2,01-10-22 5:00,448.0125,28.079184,21.250000
3,01-10-22 6:00,462.2875,28.233972,21.583333
4,01-10-22 7:00,449.2500,28.388761,21.916667


In [55]:
def preprocess_dates_string(dataframe, date_column_name):
    for i in range(len(dataframe)):
        if len(dataframe.iloc[i][0].split(' ')[-1].split(':')[0]) < 2:
            dataframe.at[i, date_column_name] = dataframe.iloc[i][0].split(' ')[0] + " 0" + \
                                                dataframe.iloc[i][0].split(' ')[-1]

    return dataframe

In [56]:
dataset = preprocess_dates_string(autumn_data, 'datetime')

/tmp/ipykernel_934/63390090.py:3: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if len(dataframe.iloc[i][0].split(' ')[-1].split(':')[0]) < 2:


In [57]:
dataset = dataset[:len(dataset) - 24 * 2]
date_time = pd.to_datetime(dataset.datetime, format='%d-%m-%y %H:%M')

In [58]:
holiday_dates = ['2022-10-28', '2022-11-11', '2022-11-17', '2022-12-25', '2022-12-26']
holiday_cal = pd.DatetimeIndex(holiday_dates)
holidays = [1 if dt.date() in holiday_cal.date or dt.dayofweek == 5 or dt.dayofweek == 6 else 0 for dt in date_time]

In [60]:
work_hour_range = range(8, 16)
work_hours = [1 if dt.time().hour in work_hour_range and holidays[num] != 1 else 0 for num, dt in enumerate(date_time)]
dataset['is_work_hour'] = work_hours

/tmp/ipykernel_934/3300493263.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset['is_work_hour'] = work_hours


In [63]:
dataset.set_index('datetime', inplace=True)
dataset = dataset[dataset['is_work_hour'] == 1].drop(['is_work_hour'], axis=1)

In [64]:
dataset.head()

,active_electricity,outdoor_temperature,indoor_temperature
datetime,,,
03-10-22 08:00,549.55,28.513594,23.250000
03-10-22 09:00,629.40,28.996465,23.583333
03-10-22 10:00,732.05,28.892873,23.916667
03-10-22 11:00,1566.00,28.587663,24.250000
03-10-22 12:00,3061.65,28.182514,24.583333


In [65]:
def prepare_timeseries_data(data):
    data['active_electricity_8'] = data.shift(8)['active_electricity'].dropna()
    return data.dropna()


input = prepare_timeseries_data(dataset)

In [66]:
input.head()

,active_electricity,outdoor_temperature,indoor_temperature,active_electricity_8
datetime,,,,
04-10-22 08:00,378.1250,26.889868,18.500000,549.55
04-10-22 09:00,687.1625,27.265097,20.500000,629.40
04-10-22 10:00,618.7375,27.271529,22.500000,732.05
04-10-22 11:00,1016.6875,27.142694,23.500000,1566.00
04-10-22 12:00,1224.0500,27.013860,24.166667,3061.65


In [67]:
X = input.copy().drop(['active_electricity'], axis=1)
y = input.copy()['active_electricity']

In [70]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=42)

In [72]:
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(401, 3) (401,)
(71, 3) (71,)


In [114]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from gplearn.genetic import SymbolicRegressor

regressor = SymbolicRegressor(
    feature_names = X_train.columns,
    random_state = 0
)

gp_model_pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('regressor', regressor)
])

In [115]:
gp_model_pipeline.fit(X_train, y_train)

Pipeline(steps=[('scaler', StandardScaler()),
                ('regressor',
                 SymbolicRegressor(feature_names=Index(['outdoor_temperature', 'indoor_temperature', 'active_electricity_8'], dtype='object'),
                                   random_state=0))])

In [116]:
y_pred = gp_model_pipeline.predict(X_test)

In [117]:
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error, mean_absolute_error

def compute_metrics(y_true, y_pred):
    mse = mean_squared_error(y_true, y_pred, squared=True)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(y_true, y_pred)
    mape = mean_absolute_percentage_error(y_true, y_pred) * 100

    return np.round(mse, 2), np.round(rmse, 2), np.round(mae, 2), np.round(mape, 2)

In [118]:
print(regressor._program)

add(sub(add(sub(add(0.005, add(mul(add(add(add(0.561, outdoor_temperature), div(0.941, 0.005)), mul(sub(div(0.941, 0.005), mul(indoor_temperature, add(div(sub(div(indoor_temperature, active_electricity_8), add(outdoor_temperature, -0.107)), add(0.431, active_electricity_8)), div(0.941, 0.005)))), sub(0.805, -0.514))), active_electricity_8), add(0.431, active_electricity_8))), add(sub(mul(outdoor_temperature, outdoor_temperature), mul(div(0.941, 0.005), sub(0.805, -0.514))), add(mul(outdoor_temperature, active_electricity_8), mul(0.928, -0.781)))), add(sub(0.805, -0.514), div(0.941, 0.005))), add(sub(mul(outdoor_temperature, outdoor_temperature), add(add(mul(div(0.941, 0.005), sub(0.805, -0.514)), div(0.941, 0.005)), add(sub(div(0.941, 0.005), add(add(active_electricity_8, -0.873), 0.941)), add(div(sub(div(indoor_temperature, active_electricity_8), add(outdoor_temperature, -0.107)), sub(add(outdoor_temperature, outdoor_temperature), add(sub(div(0.941, 0.005), mul(indoor_temperature, 0.4

In [119]:
metric = compute_metrics(y_test, y_pred)
print('MSE:',metric[0], ' RMSE:',metric[1],' MAE:',metric[2], ' MAPE%:',metric[3])

MSE: 665552.52  RMSE: 815.81  MAE: 578.27  MAPE%: 39.5


/home/rodion/miniconda3/envs/energy-data/lib/python3.10/site-packages/sklearn/metrics/_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [121]:
import pickle as pkl

X_train['y'] = y_train
X_train.to_csv("energy_train_data.csv")

X_test['y'] = y_test
X_test.to_csv("energy_test_data.csv")

with open("energy_gp_model.pkl", "wb") as f:
    pkl.dump(gp_model_pipeline, f)